In [27]:
ALL_G=[]
ALL_G_WIDTH=[]
G_short_path_ALL_list=[]
G_spatial_path_ALL_list=[]
width_G_short_path_ALL_list=[]
width_G_spatial_path_ALL_list=[]

ALL_EDGE_ARRAYS=[]
ALL_NODE_ARRAYS=[]
ALL_CH_LIST=[]
ALL_NE_LIST=[]
ALL_JU_LIST=[]
ALL_TE_LIST=[]

ALL_DFS=[]
ALL_KEYED_NODES=[]

In [28]:
ALL_DFS=[]
ALL_KEYED_NODES=[]


In [29]:
#'LUKE' OR 'SUPER'
USR='SUPER'

In [31]:
import ThreeD_Net_Tools_240223_3 as D3

import sys
import math
import numpy as np #v 1.23.3
import networkx as nx
import matplotlib.cm as cm
import matplotlib
import argparse
import matplotlib.pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from networkx.algorithms import community
import community
from community import community_louvain #conda install python-louvain
from PIL import Image
import os
import scipy
from numpy import roots 
%matplotlib notebook





#os.getcwd()
list_paths_skl=[]
list_paths_vol=[]

if USR=='LUKE':
    ALL_ANALYSIS='ALL_GAUDI.csv'#finalfilename
    directory="/media/ll16598/One Touch/CT_ANALYSIS"
    dir_G="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/BASE"
    dir_G_inverted='/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/INVERTED'
    dir_G_widths='/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS'
    dir_G_widths_inverted='/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS_INVERTED'
    dir_G_short="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SHORT"
    dir_G_spatial="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL"
    dir_G_short_widths="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SHORT_WIDTH"
    dir_G_spatial_widths="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL_WIDTH"
    
    os.chdir("/media/ll16598/One Touch/CT_ANALYSIS")
    VOXEL_SIZES = pd.read_csv('/home/ll16598/Documents/ARCHITECTURAL_IMMUNITY/ANALYSIS/VOXEL_SIZES.csv')

elif USR=='SUPER':
    directory="/media/cf19810/One Touch/CT_ANALYSIS"
    dir_G="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/BASE"
    dir_G_inverted='/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/INVERTED'
    dir_G_widths='/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS'
    dir_G_widths_inverted='/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS_INVERTED'
    dir_G_short="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SHORT"
    dir_G_spatial="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL"
    dir_G_short_widths="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SHORT_WIDTH"
    dir_G_spatial_widths="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL_WIDTH"
    
    os.chdir("/media/cf19810/One Touch/CT_ANALYSIS")
    VOXEL_SIZES = pd.read_csv('/home/cf19810/Documents/LUKE/VOXEL_SIZES.csv')

folders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f)) and "ANALYSIS" in f]

#os.chdir("/media/ll16598/One Touch/CT_ANALYSIS")


def sort_key(folder_name):
    r_number, c_number = folder_name.split("C")[0][1:], folder_name.split("C")[1][0]
    return (int(r_number), int(c_number))

analysis_folders = sorted(folders, key=sort_key)

completed_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and ".csv" in f]

completed_reps=[]
for file in completed_files:
    name=os.path.basename(file)
    name=name.replace('.csv', '')
    completed_reps.append(name)




voxel_list=list(VOXEL_SIZES['voxel'])

#NETWORK REDUCTION PARAMS
rm_loop=False #loop removal in remove_intermediate_nodes
edge_reduction=0.4
edge_pruning=1.4

#FOR TEMPORAL LINKAGE
THRSH=1.2
CH_THRSH=1.4

#NEST ENTRANCE PARAMS
D1NE=False#onlyD1 nest entrances?
NE_threshold=0.5 #cm threshold from top for d 1 nodes to be nest entrances
#now for chamers/junction
#CHAMBER PARAMS
CHAM_TUN_DIAM_ASSIGN=True #this includes if whether chambers can be assigned based on diameter compared to mean tunnel diam
#if using diam_assig, this must be specified. difference in diam compared to mean
ch_tun_thrsh=15
allowance=1
scale_factor, increment, decrement=1.5,1.2,0.8
slope_max_thresh=0.1
intensity_thresh=1.3 #how many times greater must inner intensity be compared to outer
#PARAMETERS = "THRSH=" + str(THRSH) + "\nD1NE=" + str(D1NE) + "\nNE_threshold=" + str(NE_threshold) + "\nedge_reduction=" + str(edge_reduction) + "\nedge_pruning=" + str(edge_pruning) + "\nallowance=" + str(allowance) + "\nscale_factor, increment, decrement=" + str(scale_factor) + ", " + str(increment) + ", " + str(decrement) + "\nslope_max_thresh=" + str(slope_max_thresh) + "\nintensity_thresh=" + str(intensity_thresh)ERS=
RM_JUNC=True # this removes junctions from the G analysis

skeleton_folders = []
for analysis_folder in analysis_folders:
    skeleton_folder = os.path.join(directory, analysis_folder)
    skel_folder = [os.path.join(skeleton_folder, f) for f in os.listdir(skeleton_folder) if os.path.isdir(os.path.join(skeleton_folder, f)) and "SKEL_ANALYSIS" in f]
    skeleton_folders.extend(skel_folder)

volume_folders = []
for analysis_folder in analysis_folders:
    volume_folder = os.path.join(directory, analysis_folder)
    vol_folder = [os.path.join(volume_folder, f) for f in os.listdir(volume_folder) if os.path.isdir(os.path.join(volume_folder, f)) and "VOLUMES" in f]
    volume_folders.extend(vol_folder)




In [32]:
analysis_folders

['R1C1SK_ANALYSIS',
 'R1C2PM_ANALYSIS',
 'R2C1PK_ANALYSIS',
 'R2C2SM_ANALYSIS',
 'R3C1SK_ANALYSIS',
 'R3C2PM_ANALYSIS',
 'R4C1PK_ANALYSIS',
 'R4C2SM_ANALYSIS',
 'R5C1SK_ANALYSIS',
 'R5C2PM_ANALYSIS',
 'R6C1PK_ANALYSIS',
 'R6C2SM_ANALYSIS',
 'R7C1SK_ANALYSIS',
 'R7C2PM_ANALYSIS',
 'R8C1PL_ANALYSIS',
 'R8C2SL_ANALYSIS',
 'R9C1SL_ANALYSIS',
 'R9C2PL_ANALYSIS',
 'R10C1PL_ANALYSIS',
 'R10C2SL_ANALYSIS']

In [33]:
volume_folders

#skeleton_folders


['/media/cf19810/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R1C2PM_ANALYSIS/R1C2PM_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R2C1PK_ANALYSIS/R2C1PK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R2C2SM_ANALYSIS/R2C2SM_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R3C1SK_ANALYSIS/R3C1SK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R3C2PM_ANALYSIS/R3C2PM_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R4C1PK_ANALYSIS/R4C1PK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R4C2SM_ANALYSIS/R4C2SM_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R5C1SK_ANALYSIS/R5C1SK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R5C2PM_ANALYSIS/R5C2PM_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R6C1PK_ANALYSIS/R6C1PK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R6C2SM_ANALYSIS/R6C2SM_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R7C1SK_ANALYSIS/R7C1SK_VOLUMES',
 '/media/cf19810/One Touch/CT_ANALYSIS/R7C2PM_ANALYSIS/R7C2PM_VO

In [34]:

# path = "/media/cf19810/One Touch/CT_ANALYSIS/R3C1SK_ANALYSIS/R3C1SK_SKEL_ANALYSIS/"
# path_vol = "/media/cf19810/One Touch/CT_ANALYSIS/R3C1SK_ANALYSIS/R3C1SK_VOLUMES/"

# path = "/media/ll16598/One Touch/CT_ANALYSIS/R7C2PM_ANALYSIS/R7C2PM_SKEL_ANALYSIS/"
# path_vol = "/media/ll16598/One Touch/CT_ANALYSIS/R7C2PM_ANALYSIS/R7C2PM_VOLUMES/"
# voxels=voxel_list[vs:vs+5]

vs=0
##start of loop
for fol in range(0,len(skeleton_folders)):
    
    path=skeleton_folders[fol]
    base = os.path.basename(path)
    result = os.path.splitext(base)[0].split("_")[0]
    if result in completed_reps:
        print('already obtained data for', result, 'next:')
        vs+=5
        continue
    path_vol=volume_folders[fol]
    save_dir=analysis_folders[fol]
    voxels=voxel_list[vs:vs+5]
    vs+=5

    column_names = ['day',
                    'voxels'
       ]

    analysis_df = pd.DataFrame(columns=column_names)

    analysis_df['voxels']=voxels

    skeleton_files = [ f.path for f in os.scandir(path)]

    day_list=[]
    skeleton_files1=[]
    d=0
    while d<5:
        for file in skeleton_files:
            day=os.path.basename(file)
            day=day.replace('.csv', '')
            if 'TUE' in day and d==0:
                day_list.append(day)
                skeleton_files1.append(file)
                REP_FILENAME= day[:day.index("_")]+".csv"
                d+=1
            if 'WED' in day and d==1:
                day_list.append(day)
                skeleton_files1.append(file)
                d+=1
            if 'THU' in day and d==2:
                day_list.append(day)
                skeleton_files1.append(file)
                d+=1
            if 'FRI' in day and d==3:
                day_list.append(day)
                skeleton_files1.append(file)
                d+=1
            if 'MON' in day and d==4:
                day_list.append(day)
                skeleton_files1.append(file)
                d+=1
    print(day_list)
    print(skeleton_files1)
    analysis_df['day'] = day_list

    dfs = [pd.read_csv(file, low_memory=False) for file in skeleton_files1]
    df_dict={}
    TREAT=[]
    for key, value in enumerate(dfs):
        key=day_list[key]
        df_dict[key] = value
        print("key: ", key)
        if 'P' in key:
            TREAT.append('PATHOGEN')
        else:
            TREAT.append('SHAM')
    analysis_df['treatment']=TREAT

    print('READING TIFF FILES')
    vol_tiffs = [ f.path for f in os.scandir(path_vol)]
    all_tiffs=[]
    day_list=[]
    d=0
    while d<5:
        for folder in vol_tiffs:
            day=os.path.basename(folder)
            day=day.replace('.tiff', '')
            tiff_list=[]
            stack_list=[]
            if d==0 and 'TUE' in day:
                d+=1
                vol_files = [ f.path for f in os.scandir(folder)]
                day_list.append(day)
                print(day)
                for img in vol_files:
                        name=os.path.basename(img)
                        image=Image.open(img)
                        img_name=img.replace('.tiff', '')
                        #img_data=plt.imread(image)
                        img_data=np.array(image) #may be as tiff
                        stack_list.append(img_data)
                img_stack=np.array(stack_list)
                img_stack[img_stack==0.00000000]=np.nan
                all_tiffs.append(img_stack)
                continue
            if d==1 and 'WED' in day:
                d+=1
                vol_files = [ f.path for f in os.scandir(folder)]
                day_list.append(day)
                print(day)
                for img in vol_files:
                        name=os.path.basename(img)
                        image=Image.open(img)
                        img_name=img.replace('.tiff', '')
                        #img_data=plt.imread(image)
                        img_data=np.array(image) #may be as tiff
                        stack_list.append(img_data)
                img_stack=np.array(stack_list)
                img_stack[img_stack==0.00000000]=np.nan
                all_tiffs.append(img_stack)
                continue
            if d==2 and 'THU' in day:
                d+=1
                vol_files = [ f.path for f in os.scandir(folder)]
                day_list.append(day)
                print(day)
                for img in vol_files:
                        name=os.path.basename(img)
                        image=Image.open(img)
                        img_name=img.replace('.tiff', '')
                        #img_data=plt.imread(image)
                        img_data=np.array(image) #may be as tiff
                        stack_list.append(img_data)
                img_stack=np.array(stack_list)
                img_stack[img_stack==0.00000000]=np.nan
                all_tiffs.append(img_stack)
                continue
            if d==3 and 'FRI' in day:
                d+=1
                vol_files = [ f.path for f in os.scandir(folder)]
                day_list.append(day)
                print(day)
                for img in vol_files:
                        name=os.path.basename(img)
                        image=Image.open(img)
                        img_name=img.replace('.tiff', '')
                        #img_data=plt.imread(image)
                        img_data=np.array(image) #may be as tiff
                        stack_list.append(img_data)
                img_stack=np.array(stack_list)
                img_stack[img_stack==0.00000000]=np.nan
                all_tiffs.append(img_stack)
                continue
            if d==4 and 'MON' in day:
                d+=1
                vol_files = [ f.path for f in os.scandir(folder)]
                day_list.append(day)
                print(day)
                for img in vol_files:
                        name=os.path.basename(img)
                        image=Image.open(img)
                        img_name=img.replace('.tiff', '')
                        #img_data=plt.imread(image)
                        img_data=np.array(image) #may be as tiff
                        stack_list.append(img_data)
                img_stack=np.array(stack_list)
                img_stack[img_stack==0.00000000]=np.nan
                all_tiffs.append(img_stack)
                break

    #df = pd.read_csv (r'/media/ll16598/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_SKELETON_ANALYSIS/R1C1SK_TUE.csv')

    print('REFINING EDGE LISTS')
    list_of_edge_lists=[]
    list_of_edge_lists_no_dist=[]
    list_of_node_lists=[]

    nest_mean_tun_dims=[]
    nest_std_tun_dims=[]
    nest_all_dims=[]
    all_keyed_tun_dims=[]#this is tuimportant for later if we want edge weights also based upon tunnel width

    angle_3D_means=[]
    angle_vert_means=[]
    angle_hor_means=[]
    angle_3D_stds=[]
    angle_vert_stds=[]
    angle_hor_stds=[]
    list_list_3D_angles=[]
    list_list_vert_angles=[]
    list_list_hor_angles=[]

    for df in range(0,len(dfs)):

        voxel=voxel_list[df]

        list_edge, array_edge=D3.edgez_1(dfs[df])
        list_edge=D3.tuple_convert(list_edge)

        #EDGE REDUCTION
        edge_list_reduced=D3.reduce_network_branch(list_edge, edge_reduction, 1000) 

        #EDGE PRUNING
        edge_list_pruned=D3.prune_branch(edge_list_reduced,edge_pruning)


        #TUNNEL DIMS - this is don on non-intermediate removed, since geometry is preserved more
        img_stack=all_tiffs[df]
        dims=[]
        keyed_dims=[]
        for edge in edge_list_pruned:
            img_stack=all_tiffs[2]
            dim1, dim2, mean_dim, midpoint, area=D3.tunnel_dims(edge, img_stack, voxel=voxel)
            midpoint_area=tuple([midpoint,area])
            print("tunnel dim, area=", dim1, dim2, area)
            if mean_dim is not np.nan:
                keyed_dims.append(midpoint_area)
            #print(dim1, dim2, mean_dim)
            dims.append(mean_dim)
        nest_mean_dim=np.nanmean(dims)
        nest_std_dim=np.nanstd(dims)
        nest_mean_tun_dims.append(nest_mean_dim)
        nest_std_tun_dims.append(nest_std_dim)
        nest_all_dims.append(dims)
        all_keyed_tun_dims.append(keyed_dims)
        print('THIS IS:', df)
        #INTERMEDIATE RM
        edge_list_intermediate_removed=D3.remove_intermediate_nodes(rm_loop, edge_list_pruned, 1000)
        edge_list_tup, array_edge_list1 =D3.list_convert(edge_list_intermediate_removed)
        array_edge_list2=D3.remove_length(edge_list_tup)
        list_node, array_node = D3.nodez(array_edge_list2)
        list_of_edge_lists.append(edge_list_intermediate_removed)
        list_of_edge_lists_no_dist.append(array_edge_list2)
        list_of_node_lists.append(array_node)

        #ANGLES:
        list_3D_angles, list_vertical_angles, list_horizontal_angles=D3.edge_angles(edge_list_intermediate_removed)
        angle_3D_mean=np.nanmean(list_3D_angles)
        angle_3D_std=np.nanstd(list_3D_angles)
        angle_vert_mean=np.nanmean(list_vertical_angles)
        angle_vert_std=np.nanstd(list_vertical_angles)
        angle_hor_mean=np.nanmean(list_horizontal_angles)
        angle_hor_std=np.nanstd(list_horizontal_angles)
        angle_3D_means.append(angle_3D_mean)
        angle_vert_means.append(angle_vert_mean)
        angle_hor_means.append(angle_hor_mean)
        angle_3D_stds.append(angle_3D_std)
        angle_vert_stds.append(angle_vert_std)
        angle_hor_stds.append(angle_hor_std)
        list_list_3D_angles.append(list_3D_angles)
        list_list_vert_angles.append(list_vertical_angles)
        list_list_hor_angles.append(list_horizontal_angles)
        if df==0:
            analyse_edge0=array_edge_list2
            analyse_node0=array_node
            edge_list0=edge_list_intermediate_removed
        if df==1:
            analyse_edge1=array_edge_list2
            analyse_node1=array_node
            edge_list1=edge_list_intermediate_removed
        if df==2:
            analyse_edge2=array_edge_list2
            analyse_node2=array_node
            edge_list2=edge_list_intermediate_removed
        if df==3:
            analyse_edge3=array_edge_list2
            analyse_node3=array_node
            edge_list3=edge_list_intermediate_removed
        if df==4:
            analyse_edge4=array_edge_list2
            analyse_node4=array_node
            edge_list4=edge_list_intermediate_removed
        ALL_EDGE_ARRAYS.append(array_edge_list2)
        ALL_NODE_ARRAYS.append(array_node)


    analysis_df['tunnel_dims'] = nest_mean_tun_dims
    analysis_df['tunnel_dims_std'] = nest_std_tun_dims
    analysis_df['tunnel_angle']= angle_3D_means
    analysis_df['sd_tunnel_angle']= angle_3D_std
    analysis_df['tunnel_angle_vert']= angle_vert_means
    analysis_df['sd_tunnel_angle_vert']= angle_vert_std
    analysis_df['tunnel_angle_hor']= angle_hor_means
    analysis_df['sd_tunnel_angle']= angle_hor_std

    analysis_df['edge_list']= list_of_edge_lists
    analysis_df['tunnel_dims_list']=nest_all_dims
    analysis_df['keyed_tunnel_area_list']=all_keyed_tun_dims
    analysis_df['3d_angle_list']=list_list_3D_angles
    analysis_df['vert_angle_list']=list_list_vert_angles
    analysis_df['hor_angle_list']=list_list_hor_angles

    double_edges_list=[]
    double_edges_len_list=[]
    for edge_list in list_of_edge_lists:

        double_edges, double_edges_len=D3.find_double_edge(list_of_edge_lists[4])
        double_edges_list.append(double_edges)
        double_edges_len_list.append(double_edges_len)

    analysis_df["double_edges_list"]=double_edges_list
    analysis_df["double_edges_len_list"]=double_edges_len_list

    edge_widths=[]
    for edge in list_of_edge_lists[0]:
        keyed_width, min_dist= D3.key_width(edge, all_keyed_tun_dims[0])
        edge_widths.append(tuple([edge, keyed_width, min_dist]))

    print('AUTOMATICALLY ASSIGNING NEST QUALITIES')

    ALL_NE=[]
    ALL_TE=[]
    ALL_CH=[]
    ALL_JU=[]



    for i in range(0,len(list_of_edge_lists)):
        MEAN_TUN_DIAMS=nest_mean_tun_dims[i]
        img_stack=all_tiffs[i]
        voxel=voxel_list[i]
        edge_list=list_of_edge_lists[i]
        if CHAM_TUN_DIAM_ASSIGN==False:
            nest_entrances, tunnel_ends, chambers, junctions, ellipsoid_vols=D3.assign_chambers(D1NE, edge_list, img_stack, NE_threshold, slope_max_thresh, intensity_thresh, allowance, scale_factor, increment, decrement, voxel=voxel)
        else:
            nest_entrances, tunnel_ends, chambers, junctions, ellipsoid_vols=D3.assign_chambers_2(D1NE, MEAN_TUN_DIAMS, ch_tun_thrsh, edge_list, img_stack, NE_threshold, slope_max_thresh, intensity_thresh, allowance, scale_factor, increment, decrement, voxel=voxel)

        ALL_NE.append(nest_entrances)
        ALL_TE.append(tunnel_ends)
        ALL_CH.append(chambers)
        ALL_JU.append(junctions)






    print('QUALITY ASSIGNMENT DONE. lINKING NODES AND COMPUTING Gs')
    keyed_list_of_list_nodes, keyed_edge_list_dist_all= D3.temporal_linkage(ALL_NE,ALL_TE, ALL_CH, ALL_JU, list_of_edge_lists, THRSH)
    keyed_list_of_list_nodes2, LL_CH, LL_CH_NUMS, LL_JU, LL_JU_NUMS, LL_NE, LL_NE_NUMS, LL_TE, LL_TE_NUMS=D3.temporal_linkage_2(keyed_list_of_list_nodes, ALL_CH, ALL_JU, ALL_NE, ALL_TE, list_of_edge_lists, THRSH, CH_THRSH)
    keyed_edge_list_dist_all2=D3.make_edge_list_keyed(keyed_edge_list_dist_all, keyed_list_of_list_nodes2)
    keyed_edge_list_dist_all=keyed_edge_list_dist_all2
    ALL_KEYED_NODES.append(keyed_list_of_list_nodes2)
    
    analysis_df['time_linked_edge_list']= keyed_edge_list_dist_all2
    analysis_df['chambers_list']= LL_CH
    analysis_df['end_points_list']= LL_TE
    analysis_df['junctions_list']= LL_JU
    analysis_df['nest_entrances_list']= LL_NE

    ALL_CH_LIST.append(LL_CH)
    ALL_NE_LIST.append(LL_NE)
    ALL_JU_LIST.append(LL_JU)
    ALL_TE_LIST.append(LL_TE)

    COUNT_NE=[]
    COUNT_TE=[]
    COUNT_CH=[]
    COUNT_JU=[]

    for i in range(0,len(list_of_edge_lists)):
        nest_entrances=LL_NE[i]
        tunnel_ends=LL_TE[i]
        chambers=LL_CH[i]
        junctions=LL_JU[i]

        COUNT_NE.append(len(nest_entrances))
        COUNT_TE.append(len(tunnel_ends))
        COUNT_CH.append(len(chambers))
        COUNT_JU.append(len(junctions))


    analysis_df['chambers']= COUNT_CH
    analysis_df['end_points']= COUNT_TE
    analysis_df['junctions']= COUNT_JU
    analysis_df['nest_entrances']= COUNT_NE

    print('CALCULATING CHAMBER VOLUMES')
    ALL_CHAM_VOLS=[]#chamber ellipsoid volumes
    mean_cham_vols_list=[]
    std_cham_vols_list=[]
    for ch in range(0,len(LL_CH)):

        ch_list=LL_CH[ch]
        ellipsoid_vols=D3.chamber_vols(ch_list,img_stack, scale_factor, increment, decrement,allowance,voxel=voxel)
        ALL_CHAM_VOLS.append(ellipsoid_vols)
        mean_cham_vols=np.nanmean(ellipsoid_vols)
        std_cham_vols=np.nanstd(ellipsoid_vols)
        mean_cham_vols_list.append(mean_cham_vols)
        std_cham_vols_list.append(std_cham_vols)


    analysis_df['cham_vol']= mean_cham_vols_list
    analysis_df['std_cham_vol']= std_cham_vols_list
    analysis_df['cham_vol_list']= ALL_CHAM_VOLS

    nest_ent_mean_dims=[]
    nest_ent_std_dims=[]
    nest_ent_all_dims=[]

    chamber_mean_depths=[]
    chamber_std_depths=[]
    chamber_all_depths=[]

    mean_depths=[]
    std_depths=[]
    all_depths=[]
    max_depths=[]

    for edge_list in range(0,len(list_of_edge_lists)):
        img_stack=all_tiffs[edge_list]
        dims=[]
        list_node, array_node = D3.nodez(list_of_edge_lists[edge_list])
        #NEST ENTRANCE DIAMETER
        for node in LL_NE[edge_list]:
            NE=node[0]
            x_diam, y_diam, mean_dim=D3.measure_dims_xy(NE, img_stack, voxel=voxel)
            dims.append(mean_dim)

        #CHAMBER HEIGHT
        ch_depth_list, ch_mean_depth, ch_depth_dev=D3.chamber_depth(list_node, LL_CH[edge_list])
        chamber_mean_depths.append(ch_mean_depth)
        chamber_std_depths.append(ch_depth_dev)
        chamber_all_depths.append(ch_depth_list)

        #DEPTHS
        depth_list, mean_depth, depth_dev,max_depth=D3.calculate_depth(list_node, LL_CH[edge_list])
        mean_depths.append(mean_depth)
        std_depths.append(depth_dev)
        all_depths.append(depth_list)
        max_depths.append(max_depth)


        dims=np.array(dims)
        nest_ent_mean_dim=np.nanmean(dims)
        nest_ent_std_dim=np.nanstd(dims)
        nest_ent_mean_dims.append(nest_ent_mean_dim)
        nest_ent_std_dims.append(nest_ent_std_dim)
        nest_ent_all_dims.append(dims)
        print("NEST ENTRANCE DIMS (mean/std)=",nest_ent_mean_dim, nest_ent_std_dim)
        print("DEPTHS (mean/std/max)=",mean_depth, depth_dev, max_depth)


    analysis_df['nest_entrance_dims']= nest_ent_mean_dims
    analysis_df['std_nest_entrance_dims']=nest_ent_std_dims

    analysis_df['mean_cham_depth']=chamber_mean_depths
    analysis_df['sd_cham_depth']=chamber_std_depths

    analysis_df['mean_depth']=mean_depths
    analysis_df['sd_depths']=std_depths
    analysis_df['max_depth']=max_depths

    analysis_df['nest_entrance_dims_list']= nest_ent_all_dims
    analysis_df['chamber_depth_list']=chamber_all_depths
    analysis_df['depth_list']=all_depths

    #now for tortuosity
    ALL_TORTS=[]
    mean_TORTS=[]
    sd_TORTS=[]
    ALL_PATHS=[]
    mean_PATHS=[]
    sd_PATHS=[]
    ALL_EUCS=[]
    mean_EUCS=[]
    sd_EUCS=[]
    for edge_list in list_of_edge_lists:
        tortuosities=[]
        paths=[]
        eucs=[]
        for edge in edge_list:
            path_dist=edge[2]
            euc_dist=D3.calculate_distance(edge[0], edge[1])
            tort=tuple(path_dist/euc_dist)
            tortuosities.append(tort)
            paths.append(path_dist)
            eucs.append(euc_dist)
        np_torts=np.array(tortuosities)
        ALL_TORTS.append(tortuosities)
        mean_TORTS.append(np.mean(np_torts))
        sd_TORTS.append(np.std(np_torts))
        np_paths=np.array(paths)
        ALL_PATHS.append(np_paths)
        mean_PATHS.append(np.mean(np_paths))
        sd_PATHS.append(np.std(np_paths))
        np_eucs=np.array(eucs)
        ALL_EUCS.append(np_eucs)
        mean_EUCS.append(np.mean(np_eucs))
        sd_EUCS.append(np.std(np_eucs))

    analysis_df['tortuosity']=mean_TORTS
    analysis_df['tortuosity_std']=sd_TORTS
    analysis_df['tunnel_path_mean']=mean_PATHS
    analysis_df['tunnel_path_std']=sd_PATHS
    analysis_df['tunnel_euc_mean']=mean_EUCS
    analysis_df['tunnel_euc_std']=sd_EUCS

    analysis_df['tortuosity_list']=ALL_TORTS
    analysis_df['path_list']=ALL_PATHS
    analysis_df['euclideans_list']=ALL_EUCS

    NODE_DICTS=[]
    NODE_DICTS_COORD=[]
    ALL_CHAM_NUMS=[]
    ALL_JUNC_NUMS=[]
    ALL_NE_NUMS=[]
    ALL_TE_NUMS=[]
    ALL_NE_ATTRS=[]
    ALL_END_ATTRS=[]
    ALL_CHAM_ATTRS=[]
    ALL_JUNC_ATTRS=[]
    ALL_ATTRS=[]
    TOTAL_NODES=[]
    for i in range(0,len(keyed_list_of_list_nodes2)):
        node_dict={"NEST EN":[], "END":[], "CHAM":[], "JUNC":[]}
        node_dict_coord={"NEST EN":[], "END":[], "CHAM":[], "JUNC":[]}
        TOTAL_NODES.append(len(keyed_list_of_list_nodes[i]))
        NE=LL_NE[i]
        TE=LL_TE[i]
        CH=LL_CH[i]
        JU=LL_JU[i]
        l=0
        attrs={}
        ne_attrs={}
        te_attrs={}
        ch_attrs={}
        ju_attrs={}
        chambers=[]
        juncs=[]
        nest_ens=[]
        end_points=[]
        for node1 in keyed_list_of_list_nodes2[i]:
            number=node1[1]
            node=node1[0]
            paired=0
            while paired==0:
                if node1 in NE:
                    node_dict["NEST EN"].append(number)
                    node_dict_coord["NEST EN"].append(node)
                    ne_attrs[number]='NEST EN'
                    attrs[number]='NEST EN'
                    nest_ens.append(number)
                    paired=1

                if node1 in TE:
                    node_dict["END"].append(number)
                    node_dict_coord["END"].append(node)
                    te_attrs[number]='END'
                    attrs[number]='END'
                    end_points.append(number)
                    paired=1
                if node1 in CH:
                    node_dict["CHAM"].append(number)
                    node_dict_coord["CHAM"].append(node)
                    ch_attrs[number]='CHAM'
                    attrs[number]='CHAM'
                    chambers.append(number)
                    paired=1
                if node1 in JU:
                    node_dict["JUNC"].append(number)
                    node_dict_coord["JUNC"].append(node)
                    ju_attrs[number]='JUNC'
                    attrs[number]='JUNC'
                    juncs.append(number)
                    paired=1
        NODE_DICTS.append(node_dict)
        ALL_NE_ATTRS.append(ne_attrs)
        ALL_END_ATTRS.append(te_attrs)
        ALL_CHAM_ATTRS.append(ch_attrs)
        ALL_JUNC_ATTRS.append(ju_attrs)
        ALL_ATTRS.append(attrs)
        NODE_DICTS_COORD.append(node_dict_coord)
        ALL_CHAM_NUMS.append(chambers)
        ALL_JUNC_NUMS.append(juncs)
        ALL_NE_NUMS.append(nest_ens)
        ALL_TE_NUMS.append(end_points)

    analysis_df['total_nodes']=TOTAL_NODES


    num_NE=[]
    num_TE=[]
    num_CH=[]
    num_JU=[]
    new_NE=[]
    new_TE=[]
    new_CH=[]
    new_JU=[]
    new_NODES=[]
    OLD_NODES=[]
    for i in range(0,len(NODE_DICTS)):
        num_NE.append(len(NODE_DICTS[i]['NEST EN']))
        num_TE.append(len(NODE_DICTS[i]['END']))
        num_CH.append(len(NODE_DICTS[i]['CHAM']))
        num_JU.append(len(NODE_DICTS[i]['JUNC']))
    for i in range(0,len(NODE_DICTS)):    
        if i ==0:
            new_NEs=len(NODE_DICTS[i]['NEST EN'])
            new_NE.append(len(NODE_DICTS[i]['NEST EN']))
            new_TEs=len(NODE_DICTS[i]['END'])
            new_TE.append(len(NODE_DICTS[i]['END']))
            new_CHs=len(NODE_DICTS[i]['CHAM'])
            new_CH.append(len(NODE_DICTS[i]['CHAM']))
            new_JUs=len(NODE_DICTS[i]['JUNC'])        
            new_JU.append(len(NODE_DICTS[i]['JUNC']))

            new_ns=(new_NEs+new_TEs+new_CHs+new_JUs)
            new_NODES.append(new_ns)
            continue
        else:
            new_NEs=(len(list(set(NODE_DICTS[i]['NEST EN'])-set(NODE_DICTS[i-1]['NEST EN']))))
            new_NE.append(new_NEs)
            new_TEs=(len(list(set(NODE_DICTS[i]['END'])-set(NODE_DICTS[i-1]['END']))))
            new_TE.append(new_TEs)
            new_CHs=(len(list(set(NODE_DICTS[i]['CHAM'])-set(NODE_DICTS[i-1]['CHAM']))))
            new_CH.append(new_CHs)
            new_JUs=(len(list(set(NODE_DICTS[i]['JUNC'])-set(NODE_DICTS[i-1]['JUNC']))))
            new_JU.append(new_JUs)
            new_ns=(new_NEs+new_TEs+new_CHs+new_JUs)
            new_NODES.append(new_ns)
            print('new nodes=', new_ns)
    for nodes in range(0,len(TOTAL_NODES)):
        old_nodes=TOTAL_NODES[nodes]-new_NODES[nodes]
        OLD_NODES.append(old_nodes)

    analysis_df['new_nes']=new_NE
    analysis_df['new_tes']=new_TE
    analysis_df['new_chs']=new_CH
    analysis_df['new_jus']=new_JU
    analysis_df['new_nodes']=new_NODES
    analysis_df['old_nodes']=OLD_NODES

    G_list=[]
    #ALL_just_num_edges=[]
    ALL_old_edges=[]
    for edge_list in range(0,len(keyed_edge_list_dist_all)):
        distance_connection_list, weight_list = D3.make_key(keyed_edge_list_dist_all[edge_list])
        G=nx.Graph(is_directed=False)
        old_edges=[]
        for i in range(len(distance_connection_list)):
                e1= tuple(distance_connection_list[i][0])
                key1=distance_connection_list[i][1]
                e2 = tuple(distance_connection_list[i][2])
                key2=distance_connection_list[i][3]
                d= distance_connection_list[i][4]
                #just_num_edges.append(key1, key2)
                G.add_edge(key1,key2, weight = d)#len =i[2], weight = d
                #print(e1,e2)
                ##print(key1,key2)
                if edge_list>0:
                    prev_G=G_list[edge_list-1]
                    old_edge=prev_G.has_edge(key1, key2)
                    if old_edge==True:
                        old_edges.append(tuple([key1,key2]))
        ALL_old_edges.append(old_edges)
        print('old edges=',len(old_edges), 'all edges=', len(keyed_edge_list_dist_all[edge_list]))
        print('new edges=',len(keyed_edge_list_dist_all[edge_list])-len(old_edges))
        nx.set_node_attributes(G, ALL_NE_ATTRS[edge_list], "NE")
        nx.set_node_attributes(G, ALL_END_ATTRS[edge_list], "END")
        nx.set_node_attributes(G, ALL_CHAM_ATTRS[edge_list], "CHAM")
        nx.set_node_attributes(G, ALL_JUNC_ATTRS[edge_list], "JUNC")
        nx.set_node_attributes(G, ALL_ATTRS[edge_list], "TYPE")

        name=day_list[edge_list]
        nx.write_graphml(G, dir_G+'/'+name+ "_graph.xml")
        ALL_G.append(G)
        G_list.append(G)


    day_list[0]

    #need to add attribuite colours here. This inverts edge weights, so modularity is takes it negatively
    G_list_inverted=[]
    for edge_list in range(0,len(keyed_edge_list_dist_all)):
        distance_connection_list, weight_list = D3.make_key(keyed_edge_list_dist_all[edge_list])
        G=nx.Graph(is_directed=False)
        old_edges=[]
        for i in range(len(distance_connection_list)):
                e1= tuple(distance_connection_list[i][0])
                key1=distance_connection_list[i][1]
                e2 = tuple(distance_connection_list[i][2])
                key2=distance_connection_list[i][3]
                d= 1/distance_connection_list[i][4]
                #just_num_edges.append(key1, key2)
                G.add_edge(key1,key2, weight = d)#len =i[2], weight = d
        nx.set_node_attributes(G, ALL_NE_ATTRS[edge_list], "NE")
        nx.set_node_attributes(G, ALL_END_ATTRS[edge_list], "END")
        nx.set_node_attributes(G, ALL_CHAM_ATTRS[edge_list], "CHAM")
        nx.set_node_attributes(G, ALL_JUNC_ATTRS[edge_list], "JUNC")
        nx.set_node_attributes(G, ALL_ATTRS[edge_list], "TYPE")

        name=day_list[edge_list]
        nx.write_graphml(G, dir_G_inverted+'/'+name+ "_graph_inverted.xml")

        G_list_inverted.append(G)

        #ALL_just_num_edges.append(just_num_edges)


    G_list_widths=[]
    G_list_widths_inverted=[]
    #ALL_just_num_edges=[]
    ALL_old_edges=[]
    ALL_edge_widths=[]
    for edge_list in range(0,len(keyed_edge_list_dist_all)):
        distance_connection_list, weight_list = D3.make_key(keyed_edge_list_dist_all[edge_list])
        G=nx.Graph(is_directed=False)
        G_invert=nx.Graph(is_directed=False)
        old_edges=[]
        edge_widths=[]

        for i in range(len(distance_connection_list)):
                e1= tuple(distance_connection_list[i][0])
                key1=distance_connection_list[i][1]
                e2 = tuple(distance_connection_list[i][2])
                #assigning width as a weight:
                edge=tuple([e1,e2])
                keyed_area, min_dist= D3.key_width(edge, all_keyed_tun_dims[edge_list])
                edge_widths.append(tuple([edge, keyed_width, min_dist]))
                key2=distance_connection_list[i][3]

                L=distance_connection_list[i][4]
                R=L/keyed_area
                R_inv=1/R
                #print(L,keyed_area,R)
                G.add_edge(key1,key2, weight = R)#len =i[2], weight = d
                G_invert.add_edge(key1,key2, weight = R_inv)#len =i[2], weight = d


        nx.set_node_attributes(G, ALL_NE_ATTRS[edge_list], "NE")
        nx.set_node_attributes(G, ALL_END_ATTRS[edge_list], "END")
        nx.set_node_attributes(G, ALL_CHAM_ATTRS[edge_list], "CHAM")
        nx.set_node_attributes(G, ALL_JUNC_ATTRS[edge_list], "JUNC")
        nx.set_node_attributes(G, ALL_ATTRS[edge_list], "TYPE")
        nx.set_node_attributes(G_invert, ALL_NE_ATTRS[edge_list], "NE")
        nx.set_node_attributes(G_invert, ALL_END_ATTRS[edge_list], "END")
        nx.set_node_attributes(G_invert, ALL_CHAM_ATTRS[edge_list], "CHAM")
        nx.set_node_attributes(G_invert, ALL_JUNC_ATTRS[edge_list], "JUNC")
        nx.set_node_attributes(G_invert, ALL_ATTRS[edge_list], "TYPE")
        name=day_list[edge_list]
        nx.write_graphml(G, dir_G_widths+'/'+name+ "_graph_widths.xml")
        ALL_G_WIDTH.append(G)
        name=day_list[edge_list]
        nx.write_graphml(G_invert, dir_G_widths_inverted+'/'+name+ "_graph_widths_inverted.xml")

        
        G_list_widths.append(G)
        G_list_widths_inverted.append(G_invert)
        ALL_edge_widths.append(edge_widths)
    
    #CALCULATES NEW EDGES
    new_edges = []
    total_edges=[]
    previous_edges=[]
    for g in range(0,len(G_list)):
        G=G_list[g]
        if g == 0:
            new_edges.append(len(G.edges()))
            total_edges.append(len(G.edges()))
            previous_edges.append(0)
            continue
        else:
            previous_G = G_list[g-1]
            new_edges_in_G = len(list(set(G.edges()).difference(set(previous_G.edges()))))
            new_edges.append(new_edges_in_G)
            total_edges.append(len(G.edges()))
            previous_edges.append(len(G.edges())-new_edges_in_G)

    analysis_df['new_edges']=new_edges
    analysis_df['total_edges']=total_edges
    analysis_df['old_edges']=previous_edges

        
    
    ###JUNCTION REMOVAL
    print('REMOVING JUNCTIONS')
    width_G_short_path_list=[]
    width_G_spatial_path_list=[]
    for g in range(0, len(G_list_widths)):
        G=G_list_widths[g]
        Gs = sorted(nx.connected_components(G), key=len, reverse=True)
        Gmax = G.subgraph(Gs[0])
        G=Gmax

        attributes = nx.get_node_attributes(G, 'TYPE')
        #     try:
        nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
        chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
        junction_nodes = [node for node, type_value in attributes.items() if type_value == 'JUNC']
        end_nodes = [node for node, type_value in attributes.items() if type_value == 'END']

        no_junc_nodes=list(nest_nodes+chamber_nodes+end_nodes)
        no_junc_nodes2=no_junc_nodes

        #check for completed paths
        completed_paths=[]
        completed_paths_spatial=[]

        shortest_paths_graph = nx.Graph()
        spatial_paths_graph = nx.Graph()

        for start_node in no_junc_nodes:
            for end_node in no_junc_nodes2:
                if start_node != end_node:
                    completed_paths.append(tuple([end_node, start_node]))
                    completed_paths_spatial.append(tuple([end_node, start_node]))
                    if tuple([start_node, end_node]) in completed_paths:
                        continue
                    completed_paths.append(tuple([start_node, end_node]))
                    completed_paths_spatial.append(tuple([start_node, end_node]))
                    # compute shortest path and add to shortest paths graph
                    shortest_path = nx.shortest_path(G, start_node, end_node)
                    path_spatial = nx.dijkstra_path(G, source=start_node, target=end_node, weight='weight')


                    edge_through_chamber=False
                    recent_node=start_node
                    for j in range(len(shortest_path)):
                        if shortest_path[j]==start_node:
                            continue
                        if shortest_path[j] in no_junc_nodes:
                            shortest_path1 = nx.shortest_path(G, recent_node, shortest_path[j])
                            shortest_path_length1 = sum([G[shortest_path[i]][shortest_path[i+1]]['weight'] for i in range(len(shortest_path)-1)])
                            if tuple([recent_node, shortest_path[j]]) not in completed_paths and tuple([shortest_path[j], recent_node]) not in completed_paths:                          
                                shortest_paths_graph.add_edge(recent_node, shortest_path[j], weight=shortest_path_length1)
                                #print(recent_node, shortest_path[j])
                            if shortest_path[j]==end_node and recent_node==start_node:
                                continue
                            else:
                                completed_paths.append(tuple([recent_node, shortest_path[j]]))
                                completed_paths.append(tuple([shortest_path[j], recent_node]))
                                edge_through_chamber=True

                    spatial_edge_through_chamber=False
                    recent_node=start_node
                    for j in range(len(path_spatial)):
                        if path_spatial[j]==start_node:
                            continue
                        if path_spatial[j] in no_junc_nodes:
                            path_spatial1 = nx.dijkstra_path(G, recent_node, path_spatial[j])
                            spatial_path_length1 = sum([G[path_spatial[i]][path_spatial[i+1]]['weight'] for i in range(len(path_spatial)-1)])
                            if tuple([recent_node, path_spatial[j]]) not in completed_paths_spatial and tuple([path_spatial[j], recent_node]) not in completed_paths_spatial:                          
                                spatial_paths_graph.add_edge(recent_node, path_spatial[j], weight=spatial_path_length1)
                                #print(recent_node, shortest_path[j])
                            if path_spatial[j]==end_node and recent_node==start_node:
                                continue
                            else:
                                completed_paths_spatial.append(tuple([recent_node, path_spatial[j]]))
                                completed_paths_spatial.append(tuple([path_spatial[j], recent_node]))
                                spatial_edge_through_chamber=True

                    if edge_through_chamber==False:                
                        shortest_path_length = sum([G[shortest_path[i]][shortest_path[i+1]]['weight'] for i in range(len(shortest_path)-1)])
                        shortest_paths_graph.add_edge(start_node, end_node, weight=shortest_path_length)
                        #now repeat for spatial
                    if spatial_edge_through_chamber==False:  
                        spatial_path_length = sum([G[path_spatial[i]][path_spatial[i+1]]['weight'] for i in range(len(path_spatial)-1)])
                        spatial_paths_graph.add_edge(start_node, end_node, weight=spatial_path_length)
    #                 print(spatial_path_length)
    #                 print(shortest_path_length)
        width_G_short_path_list.append(shortest_paths_graph)
        width_G_spatial_path_list.append(spatial_paths_graph)

        width_G_short_path_ALL_list.append(shortest_paths_graph)
        width_G_spatial_path_ALL_list.append(spatial_paths_graph)
        name=day_list[g]
        nx.write_graphml(shortest_paths_graph, dir_G_short_widths+'/'+name+ "_graph_short_width.xml")
        nx.write_graphml(spatial_paths_graph, dir_G_spatial_widths+'/'+name+ "_graph_spatial_width.xml")

    G_short_path_list=[]
    G_spatial_path_list=[]
    for g in range(0, len(G_list)):
        G=G_list[g]
        Gs = sorted(nx.connected_components(G), key=len, reverse=True)
        Gmax = G.subgraph(Gs[0])
        G=Gmax

        attributes = nx.get_node_attributes(G, 'TYPE')
        #     try:
        nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
        chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
        junction_nodes = [node for node, type_value in attributes.items() if type_value == 'JUNC']
        end_nodes = [node for node, type_value in attributes.items() if type_value == 'END']

        no_junc_nodes=list(nest_nodes+chamber_nodes+end_nodes)
        no_junc_nodes2=no_junc_nodes

        #check for completed paths
        completed_paths=[]
        completed_paths_spatial=[]

        shortest_paths_graph = nx.Graph()
        spatial_paths_graph = nx.Graph()

        for start_node in no_junc_nodes:
            for end_node in no_junc_nodes2:
                if start_node != end_node:
                    completed_paths.append(tuple([end_node, start_node]))
                    completed_paths_spatial.append(tuple([end_node, start_node]))
                    if tuple([start_node, end_node]) in completed_paths:
                        continue
                    completed_paths.append(tuple([start_node, end_node]))
                    completed_paths_spatial.append(tuple([start_node, end_node]))
                    # compute shortest path and add to shortest paths graph
                    shortest_path = nx.shortest_path(G, start_node, end_node)
                    path_spatial = nx.dijkstra_path(G, source=start_node, target=end_node, weight='weight')


                    edge_through_chamber=False
                    recent_node=start_node
                    for j in range(len(shortest_path)):
                        if shortest_path[j]==start_node:
                            continue
                        if shortest_path[j] in no_junc_nodes:
                            shortest_path1 = nx.shortest_path(G, recent_node, shortest_path[j])
                            shortest_path_length1 = sum([G[shortest_path[i]][shortest_path[i+1]]['weight'] for i in range(len(shortest_path)-1)])
                            if tuple([recent_node, shortest_path[j]]) not in completed_paths and tuple([shortest_path[j], recent_node]) not in completed_paths:                          
                                shortest_paths_graph.add_edge(recent_node, shortest_path[j], weight=shortest_path_length1)
                                #print(recent_node, shortest_path[j])
                            if shortest_path[j]==end_node and recent_node==start_node:
                                continue
                            else:
                                completed_paths.append(tuple([recent_node, shortest_path[j]]))
                                completed_paths.append(tuple([shortest_path[j], recent_node]))
                                edge_through_chamber=True

                    spatial_edge_through_chamber=False
                    recent_node=start_node
                    for j in range(len(path_spatial)):
                        if path_spatial[j]==start_node:
                            continue
                        if path_spatial[j] in no_junc_nodes:
                            path_spatial1 = nx.dijkstra_path(G, recent_node, path_spatial[j])
                            spatial_path_length1 = sum([G[path_spatial[i]][path_spatial[i+1]]['weight'] for i in range(len(path_spatial)-1)])
                            if tuple([recent_node, path_spatial[j]]) not in completed_paths_spatial and tuple([path_spatial[j], recent_node]) not in completed_paths_spatial:                          
                                spatial_paths_graph.add_edge(recent_node, path_spatial[j], weight=spatial_path_length1)
                                #print(recent_node, shortest_path[j])
                            if path_spatial[j]==end_node and recent_node==start_node:
                                continue
                            else:
                                completed_paths_spatial.append(tuple([recent_node, path_spatial[j]]))
                                completed_paths_spatial.append(tuple([path_spatial[j], recent_node]))
                                spatial_edge_through_chamber=True

                    if edge_through_chamber==False:                
                        shortest_path_length = sum([G[shortest_path[i]][shortest_path[i+1]]['weight'] for i in range(len(shortest_path)-1)])
                        shortest_paths_graph.add_edge(start_node, end_node, weight=shortest_path_length)
                        #now repeat for spatial
                    if spatial_edge_through_chamber==False:  
                        spatial_path_length = sum([G[path_spatial[i]][path_spatial[i+1]]['weight'] for i in range(len(path_spatial)-1)])
                        spatial_paths_graph.add_edge(start_node, end_node, weight=spatial_path_length)
    #                 print(spatial_path_length)
    #                 print(shortest_path_length)
        print('graph', shortest_paths_graph)
        G_short_path_list.append(shortest_paths_graph)
        G_spatial_path_list.append(spatial_paths_graph)
        
        G_short_path_ALL_list.append(shortest_paths_graph)
        G_spatial_path_ALL_list.append(spatial_paths_graph)
        name=day_list[g]
        nx.write_graphml(shortest_paths_graph, dir_G_short+'/'+name+ "_graph_short.xml")
        nx.write_graphml(spatial_paths_graph, dir_G_spatial+'/'+name+ "_graph_spatial.xml")

    analysis_df.to_csv(directory + "/" + REP_FILENAME, index=False)#+PARAMETERS
    ALL_DFS.append(analysis_df)



#outside indent

FINAL_DF=pd.concat(ALL_DFS)
FINAL_DF.to_csv(directory+"/"+'ALL_ANALYSIS.csv',index=False)#S+PARAMETERS

['R1C1SK_TUE', 'R1C1SK_WED', 'R1C1SK_THU', 'R1C1SK_FRI', 'R1C1SK_MON']
['/media/cf19810/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_SKEL_ANALYSIS/R1C1SK_TUE.csv', '/media/cf19810/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_SKEL_ANALYSIS/R1C1SK_WED.csv', '/media/cf19810/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_SKEL_ANALYSIS/R1C1SK_THU.csv', '/media/cf19810/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_SKEL_ANALYSIS/R1C1SK_FRI.csv', '/media/cf19810/One Touch/CT_ANALYSIS/R1C1SK_ANALYSIS/R1C1SK_SKEL_ANALYSIS/R1C1SK_MON.csv']
key:  R1C1SK_TUE
key:  R1C1SK_WED
key:  R1C1SK_THU
key:  R1C1SK_FRI
key:  R1C1SK_MON
READING TIFF FILES
R1C1SK_TUE
R1C1SK_WED
R1C1SK_THU
R1C1SK_FRI
R1C1SK_MON
REFINING EDGE LISTS
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= nan nan nan
tunnel dim, area= n

/home/cf19810/Documents/LUKE/ThreeD_Net_Tools_240223_3.py:69: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_edge = np.array(list_edge)
/home/cf19810/Documents/LUKE/ThreeD_Net_Tools_240223_3.py:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_edge_list=np.array(edge_list)


angle between edges rad, deg= 2.3122370336508635 132.4814232620434
angle between edges rad, deg= 1.4408923789774462 82.55705204797242
angle between edges rad, deg= 0.8253375335699622 47.28835734729574
angle between edges rad, deg= 2.5101684050288386 143.82205547523787
angle between edges rad, deg= 2.3122370336508635 132.4814232620434
angle between edges rad, deg= 1.651537688252052 94.62613924363527
angle between edges rad, deg= 2.2347153429505258 128.03975756419547
angle between edges rad, deg= 1.651537688252052 94.62613924363527
angle between edges rad, deg= 2.3489463250375775 134.58471072741804
angle between edges rad, deg= 1.1403963924387661 65.33990025868604
angle between edges rad, deg= 2.5101684050288386 143.82205547523787
angle between edges rad, deg= 2.3489463250375775 134.58471072741804
angle between edges rad, deg= 1.5162958112630938 86.8773504787405
angle between edges rad, deg= 1.4408923789774462 82.55705204797242
angle between edges rad, deg= 0.9762134168619667 55.93290869

angle between edges rad, deg= 1.4082123243871763 80.68462284569281
angle between edges rad, deg= 1.0947716275874981 62.72579379143156
angle between edges rad, deg= 2.7451199792173253 157.283789066193
angle between edges rad, deg= 1.7376221848468854 99.55841758002752
angle between edges rad, deg= 1.9292615953280399 110.53854698897281
angle between edges rad, deg= 2.0880519712934955 119.63656535908902
angle between edges rad, deg= 2.470324693784856 141.5391789808197
angle between edges rad, deg= 1.979902541601858 113.44005948101136
angle between edges rad, deg= 1.2147224628555375 69.59847040135921
angle between edges rad, deg= 2.8172169613002827 161.41464185517677
angle between edges rad, deg= 1.1415912755654238 65.40836201885493
angle between edges rad, deg= 1.6302590443822278 93.40696275613242
angle between edges rad, deg= 2.677189874021177 153.391680736574
angle between edges rad, deg= 2.6188934743809433 150.05154307638063
angle between edges rad, deg= 0.8460116839484568 48.4728989090

tunnel dim, area= 0.3513405 0.234227 0.051167878589757944
tunnel dim, area= 0.9201775 0.3178795 0.24352832441591568
tunnel dim, area= 0.45172349999999994 0.3513405 0.09546574125715311
tunnel dim, area= nan nan nan
tunnel dim, area= 0.43499299999999996 0.38480149999999996 0.09909310966416067
tunnel dim, area= 0.48518449999999996 0.38480149999999996 0.11195377947082379
tunnel dim, area= 0.9536385 0.5688369999999999 0.3486120879643766
tunnel dim, area= 0.301149 0.4182625 0.07688921820308418
tunnel dim, area= 0.267688 0.401532 0.06683151489274508
tunnel dim, area= 0.301149 0.401532 0.07320688966869775
tunnel dim, area= 0.234227 0.401532 0.06089582113582364
tunnel dim, area= nan nan nan
tunnel dim, area= 0.234227 0.38480149999999996 0.057543253365710596
tunnel dim, area= 0.702681 0.635759 0.2640284519282462
tunnel dim, area= 0.5688369999999999 1.6897805 0.8129152440613424
tunnel dim, area= 1.104213 1.7901634999999998 1.2567732327733563
tunnel dim, area= 0.1840355 0.4182625 0.056114290053859

...

inner mean intensity= 0.0030372031985149055 outer_mean_intensity= 0.0028481082339422356
intensity diff= 1.066393180680122
Finding center of chamber - old_node= (5.6068904999999996, 5.9707595, 5.7722855) max_dim_node= (5.706127499999999, 6.0038385, 5.9376805)
x_mean_slope= 0.03459329622279667 y_mean_slope= 0.0
slopes= 0.03459329622279667 0.0
intensity diff= 0
Finding center of chamber - old_node= (4.201033, 10.419884999999999, 4.961849999999999) max_dim_node= (4.201033, 10.419884999999999, 4.6972179999999994)
x_mean_slope= 0.08268443061331288 y_mean_slope= -0.0006370991920545987
slopes= 0.08268443061331288 -0.0006370991920545987
intensity diff= 0
Finding center of chamber - old_node= (4.184493499999999, 10.403345499999999, 4.4656649999999996) max_dim_node= (4.184493499999999, 10.403345499999999, 4.4656649999999996)
x_mean_slope= 0.02509472289472656 y_mean_slope= -0.00025891348620403335
slopes= 0.02509472289472656 -0.00025891348620403335
intensity diff= 0
Finding center of chamber - old_

inner mean intensity= 0.0025151694523727506 outer_mean_intensity= 0.0029202374562363196
intensity diff= 0.8612893609050438
Finding center of chamber - old_node= (3.0267285, 6.2023125, 3.473295) max_dim_node= (3.1755839999999997, 6.748116, 3.4071369999999996)
x_mean_slope= 0.021070067439666038 y_mean_slope= -9.281845837379548e-06
slopes= 0.021070067439666038 -9.281845837379548e-06
inner mean intensity= 0.004951905607381283 outer_mean_intensity= 0.0048987909946355095
intensity diff= 1.0108423920930567
Finding center of chamber - old_node= (2.7951754999999996, 4.63106, 1.8689634999999998) max_dim_node= (2.6793989999999996, 5.061087, 1.9351215)
x_mean_slope= 0.0 y_mean_slope= -0.00017251870536296696
slopes= 0.0 -0.00017251870536296696
inner mean intensity= 0.004339942556238633 outer_mean_intensity= 0.00313520399457116
intensity diff= 1.384261618623084
chamber= (2.793, 4.625, 1.863)
Finding center of chamber - old_node= (3.142505, 6.5992605, 3.4567555) max_dim_node= (3.0763469999999997, 6.3

inner mean intensity= 0.002063028496254574 outer_mean_intensity= 0.0023007822280423865
intensity diff= 0.8966639567665191
Finding center of chamber - old_node= (8.7748281, 9.2210058, 1.8508112) max_dim_node= (8.6922026, 9.1549054, 1.8838614)
x_mean_slope= 0.006502695851162023 y_mean_slope= -0.0001174155541083724
slopes= 0.006502695851162023 -0.0001174155541083724
inner mean intensity= 0.003631990596410429 outer_mean_intensity= 0.0033243260970359212
intensity diff= 1.0925494341992597
Finding center of chamber - old_node= (0.6444789000000001, 6.3621635, 3.2058694) max_dim_node= (0.661004, 6.4282639, 4.0486495)
x_mean_slope= 0.4660492652524777 y_mean_slope= -0.00029771052190206087
slopes= 0.4660492652524777 -0.00029771052190206087
inner mean intensity= 0.0018424048659658656 outer_mean_intensity= 0.002076960305815732
intensity diff= 0.887067923641543
Finding center of chamber - old_node= (0.7271044, 6.7257157, 2.0491124000000003) max_dim_node= (0.7271044, 6.7257157, 1.9995371000000002)
x_m

/tmp/ipykernel_1505283/1030344447.py:387: RuntimeWarning: Mean of empty slice
  mean_cham_vols=np.nanmean(ellipsoid_vols)
/home/cf19810/miniconda3/envs/LECKIE/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/cf19810/miniconda3/envs/LECKIE/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/cf19810/miniconda3/envs/LECKIE/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/cf19810/miniconda3/envs/LECKIE/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/cf19810/miniconda3/envs/LECKIE/lib/python3.9/site-packages/numpy/core/_methods.py:22

old edges= 42 all edges= 53
new edges= 11
old edges= 34 all edges= 63
new edges= 29
old edges= 45 all edges= 62
new edges= 17
REMOVING JUNCTIONS
graph Graph with 8 nodes and 28 edges
graph Graph with 15 nodes and 82 edges
graph Graph with 12 nodes and 53 edges
graph Graph with 16 nodes and 74 edges
graph Graph with 16 nodes and 71 edges


In [13]:
FINAL_DF=pd.concat(ALL_DFS)
FINAL_DF.to_csv(directory+"/"+'ALL_ANALYSIS.csv',index=False)#S+PARAMETERS

In [2]:
ALL_EDGE_ARRAYS=[]
ALL_NODE_ARRAYS=[]
ALL_CH_LIST=[]
ALL_NE_LIST=[]
ALL_JU_LIST=[]
ALL_TE_LIST=[]

ALL_DFS=[]
ALL_KEYED_NODES=[]

In [ ]:
# ALL_DFS=[]
# ALL_EDGE_ARRAYS=[]
# ALL_NODE_ARRAYS=[]
# ALL_CH_LIST=[]
# ALL_NE_LIST=[]
# ALL_JU_LIST=[]
# ALL_TE_LIST=[]

In [1]:
npALL_CH=[]
npALL_TE=[]
npALL_JU=[]
npALL_NE=[]

for g1 in range(0,len(ALL_CH_LIST)):
    for g in range(0,len(ALL_CH_LIST[g1])):
        XYZ_CH=[]
        XYZ_TE=[]
        XYZ_JU=[]
        XYZ_NE=[]
        for edge in ALL_CH_LIST[g1][g]:
            XYZ_CH.append(edge[0])
        for edge in ALL_TE_LIST[g1][g]:
            XYZ_TE.append(edge[0])
        for edge in ALL_JU_LIST[g1][g]:
            XYZ_JU.append(edge[0])
        for edge in ALL_NE_LIST[g1][g]:
            XYZ_NE.append(edge[0])
        npALL_CH.append(np.array(XYZ_CH))
        npALL_TE.append(np.array(XYZ_TE))
        npALL_JU.append(np.array(XYZ_JU))
        npALL_NE.append(np.array(XYZ_NE))

NameError: name 'ALL_CH_LIST' is not defined

In [2]:
# Create the 3D figure
index=54
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection="3d")
analyse_edge, analyse_node=ALL_EDGE_ARRAYS[index], ALL_NODE_ARRAYS[index]
# Plot the nodes - alpha is scaled by "depth" automatically
# npALL_NE=[]
# npALL_TE=[]
# npALL_CH=[]
# npALL_JU=[]
#ax.scatter(*analyse_node.T, s=60, ec="w", color="orange")
try:
    ax.scatter(*npALL_CH[index].T, s=60, ec="w", color="blue")
except TypeError:
    pass
ax.scatter(*npALL_TE[index].T, s=60, ec="w", color="red")
ax.scatter(*npALL_JU[index].T, s=60, ec="w", color="orange")
ax.scatter(*npALL_NE[index].T, s=60, ec="w", color="green")

for vizedge in analyse_edge: #need dist remove
    ax.plot(*vizedge.T, color="white")
    ax.set_facecolor('black')
k=0
# for i in analyse_node:
#     #ax.text(x=i[0],y=i[1],z=i[2],s=i, size=8, horizontalalignment='left')
#     ax.text(x=i[0],y=i[1],z=i[2],s=keyed_list_of_list_nodes[index][k][1], size=12, horizontalalignment='left')
#     k+=1

def format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(False)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x", color = 'white')
    ax.set_ylabel("y", color = 'white')
    ax.set_zlabel("z", color = 'white')
    #change subplot pane color
    #ax.w_xaxis.set_pane_color(('black'))
    #ax.w_yaxis.set_pane_color(('black'))
    #ax.w_zaxis.set_pane_color(('black'))

format_axes(ax)
fig.tight_layout()
plt.show()


NameError: name 'plt' is not defined